# Implementation of The Shuffled Frog Leaping Algorithm (SFLA) for Test Functions :

In [1]:
import numpy as np

In [10]:
def gen_frogs(frogs, dimension):
    frogs = np.random.uniform(-5, 5, (frogs, dimension))
    # frogs = np.random.randn(frogs, dimension)
    return frogs

# Sorts the frogs in decending order of fitness and create the memeplexes
def sort_frogs(frogs, mplx_no, opt_func):
    # Find fitness of each frog
    fitness = np.array(list(map(opt_func, frogs)))
    # Sort the indices in decending order by fitness
    sorted_fitness = np.argsort(fitness)
    # Empty holder for memeplexes
    memeplexes = np.zeros((mplx_no, int(frogs.shape[0]/mplx_no)))
    # Sort into memeplexes
    for j in range(memeplexes.shape[1]):
        for i in range(mplx_no):
            memeplexes[i, j] = sorted_fitness[i+(mplx_no*j)]
    return memeplexes

#Performs the local search for a memeplex.
def local_search(frogs, memeplex, opt_func):

    # Select worst, best, greatest frogs
    frog_w = frogs[int(memeplex[-1])]
    frog_b = frogs[int(memeplex[0])]
    frog_g = frogs[0]
    # Move worst wrt best frog
    frog_w_new = frog_w + (np.random.rand() * (frog_b - frog_w))
    # If change not better, move worst wrt greatest frog
    if opt_func(frog_w_new) > opt_func(frog_w):
        frog_w_new = frog_w + (np.random.rand() * (frog_g - frog_w))
    # If change not better, random new worst frog
    if opt_func(frog_w_new) > opt_func(frog_w):
        frog_w_new = gen_frogs(1, frogs.shape[1])[0]
    # Replace worst frog
    frogs[int(memeplex[-1])] = frog_w_new
    return frogs

# Shuffles the memeplexes without sorting them.
def shuffle_memeplexes(memeplexes):
    # Flatten the array
    temp = memeplexes.flatten()
    #Shuffle the array
    np.random.shuffle(temp)
    # Reshape
    temp = temp.reshape((memeplexes.shape[0], memeplexes.shape[1]))
    return temp

# Performs the Shuffled Leaping Frog Algorithm.
def sfla(opt_func, frogs=30, dimension=2, mplx_no=5, mplx_iters=10, solun_iters=50):
    """ - opt_func
        - frogs {int} : The number of frogs to use 
        - dimension {int} : The dimension/number of features
        - mplx_no {int} : Number of memeplexes, when divides frog number should return an integer otherwise frogs will be skipped
        - mplx_iters {int} : Number of times a single memeplex will be iterated before shuffling
        - solun_iters {int} : Number of times the memeplexes will be shuffled
    """
    # Generate frogs around the solution
    frogs = gen_frogs(frogs, dimension)
    # Arrange frogs and sort into memeplexes
    memeplexes = sort_frogs(frogs, mplx_no, opt_func)
    # Best solution as greatest frog
    best_solun = frogs[int(memeplexes[0, 0])]
    
    # For the number of iterations
    for i in range(solun_iters):
        # Shuffle memeplexes
        memeplexes = shuffle_memeplexes(memeplexes)
        
        # For each memeplex
        for mplx_idx, memeplex in enumerate(memeplexes):
            
            # For number of memeplex iterations
            for j in range(mplx_iters):
                # Perform local search
                frogs = local_search(frogs, memeplex, opt_func)
                
            # Rearrange memeplexes
            memeplexes = sort_frogs(frogs, mplx_no, opt_func)
            # Check and select new best frog as the greatest frog
            new_best_solun = frogs[int(memeplexes[0, 0])]
            if opt_func(new_best_solun) < opt_func(best_solun):
                best_solun = new_best_solun
                
    return best_solun
    # return best_solun, frogs, memeplexes.astype(int)


In [ ]:
def show_plot(x, y, title="", x_label= "", y_label="") :
    plt.plot(x, y)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

In [11]:
def rosenbrock_function(x):
    a = 1
    b = 100
    return sum(b * (x[1:] - x[:-1]**2)**2 + (a - x[:-1])**2)

def bent_cigar_function(x):
    if isinstance(x, list):
        x = np.array(x)
    return x[0]**2 + 1e6 * np.sum(x[1:]**2)

def discus_function(x):
    if isinstance(x, list):
        x = np.array(x)
    return 1e6 * x[0]**2 + np.sum(x[1:]**2)

def rastrigin_function(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

In [13]:
# Run algorithm
sol = sfla(bent_cigar_function, 1000, 2, 20, 25, 1000)
print(f"Optimal Solution: {sol}")
print(f"Optimal value : {bent_cigar_function(sol)}")

Optimal Solution: [-4.04717166e-02  2.47570578e-05]
Optimal value : 0.0022508717522394294


In [23]:
# Run algorithm
sol = sfla(rosenbrock_function, 500, 2, 30, 30, 500)
print(f"Optimal Solution: {sol}")
print(f"Optimal value : {rosenbrock_function(sol)}")

Optimal Solution: [1.03912991 1.07998375]
Optimal value : 0.001534866419944733


In [21]:
# Run algorithm
sol = sfla(discus_function, 500, 2, 30, 25, 500)
print(f"Optimal Solution: {sol}")
print(f"Optimal value : {discus_function(sol)}")

Optimal Solution: [1.60155698e-06 7.40650563e-02]
Optimal value : 0.005488197549930775


In [35]:
# Run algorithm
sol = sfla(rastrigin_function, 500, 2, 10, 30, 1000)
print(f"Optimal Solution: {sol}")
print(f"Optimal value : {rastrigin_function(sol)}")

Optimal Solution: [1 0]
Optimal value : 1.0


## SFLA for TSP :

In [33]:
def gen_frogs(frogs, dimension):
    population = []
    for _ in range(frogs):
        frog = np.random.permutation(dimension)
        population.append(frog)
    return np.array(population)

# Calculate the total distance of a tour
def calculate_distance(tour, distance_matrix):
    distance = 0
    for i in range(len(tour) - 1):
        distance += distance_matrix[tour[i]][tour[i + 1]]
    distance += distance_matrix[tour[-1]][tour[0]]  # Return to starting city
    return distance

# Sorts the frogs in descending order of fitness
def sort_frogs(frogs, mplx_no, opt_func):
    fitness = np.array(list(map(opt_func, frogs)))
    sorted_fitness = np.argsort(fitness)
    memeplexes = np.zeros((mplx_no, int(frogs.shape[0] / mplx_no)), dtype=int)
    for j in range(memeplexes.shape[1]):
        for i in range(mplx_no):
            memeplexes[i, j] = sorted_fitness[i + (mplx_no * j)]
    return memeplexes

# Performs the local search for a memeplex
def local_search(frogs, memeplex, opt_func):
    frog_w = frogs[int(memeplex[-1])]
    frog_b = frogs[int(memeplex[0])]
    frog_g = frogs[0]

    def swap_cities(frog):
        new_frog = frog.copy()
        i, j = np.random.choice(len(frog), 2, replace=False)
        new_frog[i], new_frog[j] = new_frog[j], new_frog[i]
        return new_frog

    frog_w_new = swap_cities(frog_w)
    if opt_func(frog_w_new) > opt_func(frog_w):
        frog_w_new = swap_cities(frog_w)
    if opt_func(frog_w_new) > opt_func(frog_w):
        frog_w_new = gen_frogs(1, len(frog_w))[0]
    frogs[int(memeplex[-1])] = frog_w_new
    return frogs

# Shuffles the memeplexes without sorting them
def shuffle_memeplexes(memeplexes):
    temp = memeplexes.flatten()
    np.random.shuffle(temp)
    temp = temp.reshape((memeplexes.shape[0], memeplexes.shape[1]))
    return temp

# SFLA for TSP
def sfla(opt_func, frogs=30, dimension=2, mplx_no=5, mplx_iters=10, solun_iters=50):
    frogs = gen_frogs(frogs, dimension)
    memeplexes = sort_frogs(frogs, mplx_no, opt_func)
    best_solun = frogs[int(memeplexes[0, 0])]
    for i in range(solun_iters):
        memeplexes = shuffle_memeplexes(memeplexes)
        for mplx_idx, memeplex in enumerate(memeplexes):
            for j in range(mplx_iters):
                frogs = local_search(frogs, memeplex, opt_func)
            memeplexes = sort_frogs(frogs, mplx_no, opt_func)
            new_best_solun = frogs[int(memeplexes[0, 0])]
            if opt_func(new_best_solun) < opt_func(best_solun):
                best_solun = new_best_solun
    return best_solun

# Example usage
# Example usage
cities = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"]

distance_matrix = {
    "A": {"A": 0, "B": 2, "C": 3, "D": 1, "E": 5, "F": 8, "G": 7, "H": 3, "I": 4, "J": 6, "K": 5, "L": 9},
    "B": {"A": 2, "B": 0, "C": 2, "D": 4, "E": 6, "F": 3, "G": 5, "H": 7, "I": 3, "J": 5, "K": 8, "L": 6},
    "C": {"A": 3, "B": 2, "C": 0, "D": 3, "E": 4, "F": 6, "G": 8, "H": 2, "I": 5, "J": 7, "K": 6, "L": 4},
    "D": {"A": 1, "B": 4, "C": 3, "D": 0, "E": 2, "F": 7, "G": 6, "H": 4, "I": 8, "J": 6, "K": 3, "L": 5},
    "E": {"A": 5, "B": 6, "C": 4, "D": 2, "E": 0, "F": 1, "G": 3, "H": 6, "I": 7, "J": 5, "K": 4, "L": 8},
    "F": {"A": 8, "B": 3, "C": 6, "D": 7, "E": 1, "F": 0, "G": 2, "H": 5, "I": 4, "J": 8, "K": 7, "L": 6},
    "G": {"A": 7, "B": 5, "C": 8, "D": 6, "E": 3, "F": 2, "G": 0, "H": 4, "I": 6, "J": 3, "K": 5, "L": 7},
    "H": {"A": 3, "B": 7, "C": 2, "D": 4, "E": 6, "F": 5, "G": 4, "H": 0, "I": 8, "J": 2, "K": 7, "L": 3},
    "I": {"A": 4, "B": 3, "C": 5, "D": 8, "E": 7, "F": 4, "G": 6, "H": 8, "I": 0, "J": 1, "K": 2, "L": 5},
    "J": {"A": 6, "B": 5, "C": 7, "D": 6, "E": 5, "F": 8, "G": 3, "H": 2, "I": 1, "J": 0, "K": 4, "L": 6},
    "K": {"A": 5, "B": 8, "C": 6, "D": 3, "E": 4, "F": 7, "G": 5, "H": 7, "I": 2, "J": 4, "K": 0, "L": 2},
    "L": {"A": 9, "B": 6, "C": 4, "D": 5, "E": 8, "F": 6, "G": 7, "H": 3, "I": 5, "J": 6, "K": 2, "L": 0}
}

distance_matrix = np.array([[distance_matrix[c1][c2] for c2 in cities] for c1 in cities])

def tsp_distance(tour):
    return calculate_distance(tour, distance_matrix)

best_solution= sfla(tsp_distance, frogs=50, dimension=len(cities), mplx_no=3, mplx_iters=15, solun_iters=100)

# Convert best_tour_indices to city names
best_tour = [cities[idx] for idx in best_solution]

print("Best solution:", best_tour)
print("Best distance:", tsp_distance(best_solution))


Best solution: ['L', 'H', 'B', 'F', 'E', 'G', 'A', 'D', 'C', 'J', 'I', 'K']
Best distance: 40
